In [7]:
from transformers import pipeline
from utils import *
import glob
import numpy as np
import time
import cv2

In [8]:
# More models in the model hub.
model_name = "openai/clip-vit-large-patch14-336"
classifier = pipeline("zero-shot-image-classification", model = model_name, device=0)

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [9]:
def classify(image,labels):
    start_t = time.time_ns()
    scores = classifier(image, 
                        candidate_labels = labels)
    end_t = time.time_ns()
    
    latency = (end_t-start_t)/1e6
    
    print(scores,latency)
    return scores,latency

In [10]:
protocol = "respiratory_distress_protocol"
# protocol = "adult_cardiact_arrest_protocol"

labels_for_classification = generate_labels(protocol)

# assign directory
directory = './images/resp - assist ventilation - bvm via mask/'

accuracy = []
# iterate over files in
# that directory
for filename in glob.iglob(f'{directory}/*.jpg'):
    model_scores,model_latency = classify(filename,labels_for_classification)
    accuracy.append( {"image":filename, "results": model_scores, "latency": model_latency})

[{'score': 0.7587544918060303, 'label': 'Placing Oxygen mask on face'}, {'score': 0.11074140667915344, 'label': 'Inserting IV to arm'}, {'score': 0.09729040414094925, 'label': 'Attaching nebulizer'}, {'score': 0.019345950335264206, 'label': 'BVM Mask'}, {'score': 0.008202722296118736, 'label': 'Inserting IV to leg'}, {'score': 0.005665151868015528, 'label': 'Nebulizer mask'}] 57.439214
[{'score': 0.9793322086334229, 'label': 'Placing Oxygen mask on face'}, {'score': 0.01286565326154232, 'label': 'Attaching nebulizer'}, {'score': 0.0031559744384139776, 'label': 'Inserting IV to arm'}, {'score': 0.0027314245235174894, 'label': 'Nebulizer mask'}, {'score': 0.0017531769117340446, 'label': 'BVM Mask'}, {'score': 0.00016158254584297538, 'label': 'Inserting IV to leg'}] 54.080937
[{'score': 0.9450276494026184, 'label': 'Placing Oxygen mask on face'}, {'score': 0.02062266692519188, 'label': 'Attaching nebulizer'}, {'score': 0.01985260844230652, 'label': 'BVM Mask'}, {'score': 0.007208387367427

In [11]:
avg_accuracy = []
avg_latency = []
for result in accuracy:
    print('Image: ',result['image'],'Latency(ms)',result['latency'],'Scores:',result['results'][0], )
    avg_accuracy.append(result['results'][0]['score'])
    avg_latency.append(result['latency'])
    
print("Average Accuracy:",np.average(avg_accuracy))
print("Average Latency(s):",np.average(avg_latency)/1e3)

Image:  ./images/resp - assist ventilation - bvm via mask/assist_bvm_4.jpg Latency(ms) 57.439214 Scores: {'score': 0.7587544918060303, 'label': 'Placing Oxygen mask on face'}
Image:  ./images/resp - assist ventilation - bvm via mask/assist_bvm_2.jpg Latency(ms) 54.080937 Scores: {'score': 0.9793322086334229, 'label': 'Placing Oxygen mask on face'}
Image:  ./images/resp - assist ventilation - bvm via mask/assist_bvm_3.jpg Latency(ms) 53.133477 Scores: {'score': 0.9450276494026184, 'label': 'Placing Oxygen mask on face'}
Image:  ./images/resp - assist ventilation - bvm via mask/assist_bvm_1.jpg Latency(ms) 44.490524 Scores: {'score': 0.8624255657196045, 'label': 'Placing Oxygen mask on face'}
Image:  ./images/resp - assist ventilation - bvm via mask/assist_bvm_6.jpg Latency(ms) 43.117815 Scores: {'score': 0.9254555702209473, 'label': 'Attaching nebulizer'}
Image:  ./images/resp - assist ventilation - bvm via mask/assist_bvm_5.jpg Latency(ms) 45.092212 Scores: {'score': 0.9027846455574036

Real-time classification

In [12]:
from skimage.metrics import structural_similarity
from PIL import Image

In [14]:

# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture('./datasets/oxygen-bvm/14-08-2023-14-07-44/videodata/newvideo.avi')
 
# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")
 
frame = np.array([])
fps = (cap.get(cv2.CAP_PROP_FPS))

idx = 0

# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  before = frame
  
  ret, frame = cap.read()
  if ret == True:
 
    
    scale_percent = 50 # percent of original size
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    dim = (width, height)
    
      
    # resize image
    frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
    
    after = frame
    
    
    if before.size != 0 :
      
      start_t = time.time_ns()
      before_gray = cv2.cvtColor(before, cv2.COLOR_BGR2GRAY)

      # Convert images to grayscale
      after_gray = cv2.cvtColor(after, cv2.COLOR_BGR2GRAY)

      # # Compute SSIM between two images
      (score, diff) = structural_similarity(before_gray, after_gray, full=True)
      
      

      
      end_t = time.time_ns()
      
      # print("latency (ms)", (end_t-start_t)/1e6)
      print("Image similarity, Latency (ms)", score, (end_t-start_t)/1e6)
    
      if(score < 0.90):
        # Display the resulting frame
        pil_image = Image.fromarray(after_gray)
        
        model_scores,model_latency = classify(pil_image,labels_for_classification)
        
        # cv2.imwrite(f'./images/cv - ecg - 12 lead obtained/video_extract/img_{idx}.jpg',frame)
        
        idx += 1
        
        cv2.imshow('Frame',frame)
  
    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
 
  # Break the loop
  else: 
    break
 
# When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()

Image similarity, Latency (ms) 0.7801963721972904 4.262176
[{'score': 0.6469293236732483, 'label': 'Placing Oxygen mask on face'}, {'score': 0.27827268838882446, 'label': 'Attaching nebulizer'}, {'score': 0.03870851919054985, 'label': 'Inserting IV to arm'}, {'score': 0.025691568851470947, 'label': 'Nebulizer mask'}, {'score': 0.006813641171902418, 'label': 'BVM Mask'}, {'score': 0.0035843374207615852, 'label': 'Inserting IV to leg'}] 63.639982
Image similarity, Latency (ms) 0.914154038340304 12.353639


/home/kesharaw/anaconda3/envs/emsvisionv2/lib/python3.8/site-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Image similarity, Latency (ms) 0.6498264807319987 13.285888
[{'score': 0.46264714002609253, 'label': 'Nebulizer mask'}, {'score': 0.33780720829963684, 'label': 'Placing Oxygen mask on face'}, {'score': 0.16316357254981995, 'label': 'BVM Mask'}, {'score': 0.032088082283735275, 'label': 'Attaching nebulizer'}, {'score': 0.002888657385483384, 'label': 'Inserting IV to arm'}, {'score': 0.0014054246712476015, 'label': 'Inserting IV to leg'}] 46.049754
Image similarity, Latency (ms) 0.963987789381855 3.297104
Image similarity, Latency (ms) 0.901371429848745 10.981921
Image similarity, Latency (ms) 0.9423817979076987 14.619872
Image similarity, Latency (ms) 0.9197978843864886 13.559945
Image similarity, Latency (ms) 0.936557204848781 5.169055
Image similarity, Latency (ms) 0.9535848909530747 14.462398
Image similarity, Latency (ms) 0.9627674049632389 14.387988
Image similarity, Latency (ms) 0.9776951450740108 13.286474
Image similarity, Latency (ms) 0.9850205870933645 11.982286
Image similari

In [3]:
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from scipy.spatial.distance import cosine
import keras.utils as image
import time

# Load the pre-trained ResNet50 model (weights will be downloaded automatically)
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

def extract_features(image_path, model):
    x = preprocess_image(image_path)
    features = model.predict(x)
    return features.flatten()

def compute_image_similarity(image_path1, image_path2):
    features1 = extract_features(image_path1, model)
    features2 = extract_features(image_path2, model)
    similarity = 1 - cosine(features1, features2)
    return similarity

if __name__ == "__main__":
    image_path1 = "./images/cv - ecg - 12 lead obtained/ecg_1.jpg"
    image_path2 = "./images/cv - ecg - 12 lead obtained/ecg_1.jpg"

    start_t = time.time_ns()
    similarity_score = compute_image_similarity(image_path1, image_path2)
    end_t = time.time_ns()
    
    print(f"Similarity score: {similarity_score:.4f}, latency: {(end_t-start_t)/1e6}")

1/1 [==============================] - 0s 12ms/step
Similarity score: 1.0000, latency: 363.454001


EMS Procedure Engine

In [ ]:
def detect_procedure(image, protocol, classifier):
    
    labels_for_classification = generate_labels(protocol)
    scores = classifier(image, 
                    candidate_labels = labels_for_classification)
    
    
    
    
